In [ ]:
#https://github.com/fernandoandreotti/cinc-challenge2017

63

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#cd "/content/drive/MyDrive/Colab Notebooks/ECG"


/content/drive/MyDrive/Colab Notebooks/ECG


In [ ]:
# Common imports 
import os
import numpy as np
import pandas as pd
#to unzip files: with ZipFile('sampleDir.zip', 'r') as zipObj: zipObj.extractall('temp')
import zipfile as zp
#Visualization
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#gdown is useful to download files from public files in Google Drive. --id is the id of the file
!gdown --id 1j7Yk0-W9uonQiyyG2B9ZAw5jjZA-UeMS

Downloading...
From: https://drive.google.com/uc?id=1j7Yk0-W9uonQiyyG2B9ZAw5jjZA-UeMS
To: /content/drive/MyDrive/Colab Notebooks/ECG/challenge2017.pkl
100% 167M/167M [00:01<00:00, 102MB/s] 


In [ ]:
import pickle
data_dict = pickle.load(open("challenge2017.pkl","rb"))

In [ ]:
print(data_dict["data"][0])

[-127 -162 -197 ...  -18  -22  -21]


In [ ]:
data = data_dict["data"]
labels = data_dict['label']
df = pd.DataFrame(data,columns=["ts"])\
      .assign(label = pd.Series(labels).replace({"N":"Normal","A":"Atrial Fibrillation","O":"Other Rhythm","~":"Noise"}))
df

,ts,label
0,"[-127, -162, -197, -229, -245, -254, -261, -26...",Normal
1,"[128, 157, 189, 226, 250, 257, 262, 265, 268, ...",Normal
2,"[56, 73, 85, 93, 100, 107, 113, 117, 118, 117,...",Normal
3,"[519, 619, 723, 827, 914, 956, 955, 934, 920, ...",Atrial Fibrillation
4,"[-188, -239, -274, -316, -356, -374, -380, -38...",Atrial Fibrillation
...,...,...
8523,"[-104, -127, -144, -154, -160, -164, -168, -17...",Normal
8524,"[-121, -197, -288, -390, -496, -600, -693, -77...",Other Rhythm
8525,"[-202, -235, -272, -305, -325, -339, -344, -34...",Normal
8526,"[-227, -297, -374, -449, -505, -527, -539, -54...",Normal


In [ ]:
FS = 300
maxlen = 30*FS

def preprocessing(data):
  #np.concatenate(df["ts"]).ravel().tolist())
  data = np.array([data])
  X = np.zeros((1,maxlen))
  data = np.nan_to_num(data) # removing NaNs and Infs
  data = data[0,0:maxlen]
  data = data - np.mean(data)
  data = data/np.std(data)
  X[0,:len(data)] = data.T # padding sequence
  data = X
  data = np.expand_dims(data, axis=2)
  return data

df['ts'] = df['ts'].apply(lambda x: preprocessing(x))

In [ ]:
display(df)

,ts,label
0,"[[[-0.8648478526564592], [-1.072787609234484],...",Normal
1,"[[[0.6043533556477505], [0.7685191341883563], ...",Normal
2,"[[[0.35378472457266574], [0.45487683495531683]...",Normal
3,"[[[2.239756323370247], [2.707878450360614], [3...",Atrial Fibrillation
4,"[[[-0.9897166782739075], [-1.22892495258504], ...",Atrial Fibrillation
...,...,...
8523,"[[[-1.8868172106680006], [-2.2717874413895096]...",Normal
8524,"[[[-0.17081191745581825], [-0.4113088865048658...",Other Rhythm
8525,"[[[-1.3389800883138374], [-1.5442797187476396]...",Normal
8526,"[[[-1.1545601145951963], [-1.5064293735015712]...",Normal


In [ ]:
from keras.models import load_model
model = load_model('ResNet_30s_34lay_16conv.hdf5')

classes = ['Atrial Fibrillation', 'Normal', 'Other Rhythm','Noise']

confusion_matrix = {
    'Atrial Fibrillation': [0, 0, 0, 0],
    'Normal': [0, 0, 0, 0],
    'Other Rhythm': [0, 0, 0, 0],
    'Noise': [0, 0, 0, 0]
    
}

for i in range(len(df)):   
  prob = model.predict(df["ts"][i])
  ann = np.argmax(prob)
  true_target = df["label"][i]
  #print(true_target,ann)

  confusion_matrix[true_target][ann] += 1

  #print("Record {} classified as {} with {:3.1f}% certainty. True: {}".format(i,classes[ann],100*prob[0,ann],df["label"][i]))


confusion_matrix = pd.DataFrame(confusion_matrix,index = classes)
confusion_matrix.columns = [i + " T" for i in classes]
print(confusion_matrix)

                     Atrial Fibrillation T  Normal T  Other Rhythm T  Noise T
Atrial Fibrillation                    447        23             162        3
Normal                                  18      3886             468        3
Other Rhythm                           204       504            1488       13
Noise                                   89       663             297      260


In [ ]:
accuracy = sum([confusion_matrix.iloc[i,i] for i in range(4)])/len(df)
print(f"Total accuracy of {round(accuracy*100,2)}%")

Total accuracy of 71.31%


In [ ]:
!pip install streamlit

In [ ]:
import streamlit as st
import pandas as pd
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier

st.write("""
# Simple Iris Flower Prediction App
This app predicts the **Iris flower** type!
""")

st.sidebar.header('User Input Parameters')

def user_input_features():
    sepal_length = st.sidebar.slider('Sepal length', 4.3, 7.9, 5.4)
    sepal_width = st.sidebar.slider('Sepal width', 2.0, 4.4, 3.4)
    petal_length = st.sidebar.slider('Petal length', 1.0, 6.9, 1.3)
    petal_width = st.sidebar.slider('Petal width', 0.1, 2.5, 0.2)
    data = {'sepal_length': sepal_length,
            'sepal_width': sepal_width,
            'petal_length': petal_length,
            'petal_width': petal_width}
    features = pd.DataFrame(data, index=[0])
    return features

df = user_input_features()

st.subheader('User Input parameters')
st.write(df)

iris = datasets.load_iris()
X = iris.data
Y = iris.target

clf = RandomForestClassifier()
clf.fit(X, Y)

prediction = clf.predict(df)
prediction_proba = clf.predict_proba(df)

st.subheader('Class labels and their corresponding index number')
st.write(iris.target_names)

st.subheader('Prediction')
st.write(iris.target_names[prediction])
#st.write(prediction)

st.subheader('Prediction Probability')
st.write(prediction_proba)